# Kirjavigade tabeli analüüs

Korpuses eksisteerivate lemmade põhjal moodustatud kirjavigade tabel võib seada ühele kirjaveaga vormile vastavusse mitu korrektset vormi. Mõnikord on korrektsete vormide arv liiga suur. Järgnevas analüüsime seda täpsemalt. Seda sammu pole otseselt vaja, aga see võimaldab tuvastada ootamatusi.

In [1]:
import sqlite3

from pandas import DataFrame

## I. Kirjaveale vastavate õigete vormide jaotus

Järgnevas tabelis näitab esimene rida üheselt lahenduvate kirjaveavormide arvu, kus igale kirjaveale vastab vaid üks korrektne vorm.
Teise veeru summa näitab ära kirjaveavormide koguarvu. Kirjaveale vastavate vormide peaks olema mõistlik (alla 100), muidu võib päringulaiendaja anda liiga pika vastuse tagasi.

In [2]:
con = sqlite3.connect('../results/query_extender_setup/rt_titles.sqlite')
cur = con.cursor()

In [63]:
result = cur.execute(
    """
    SELECT
        lemmade_arv,
        count(*) as kirjaveavormide_arv
    FROM
    (	
        SELECT 
            vigane_vorm, 
            count(*) as lemmade_arv
        FROM kirjavead
        GROUP By vigane_vorm
    )
    GROUP BY lemmade_arv    
    """
)
summary = (DataFrame(result.fetchall(), columns = ['vastete_arv', 'kirjaveavormide_arv'])
           .assign(osakaal_protsentides=lambda df: df['kirjaveavormide_arv']/sum(df['kirjaveavormide_arv'])*100).round(2))
display(summary)

,vastete_arv,kirjaveavormide_arv,osakaal_protsentides
0,1,3149,99.72
1,2,9,0.28


## II. Mitmeste vastete täpsem analüüs

Mitmesele kirjavea vormile vastavate õigete vormide loetelu. 
Sealjuures eemaldame mitmesused, mis on tingitud täheortograafiast (TÜRGI vs Türgi vs türgi).  

In [59]:
result = cur.execute(
    """
    SELECT
        vigane_vorm, 
        vorm, 
        kaal
    FROM kirjavead
    WHERE vigane_vorm in
    ( 
        SELECT vigane_vorm
        FROM
        (
            SELECT 
                vigane_vorm, 
                count(*) as count
            FROM kirjavead
            GROUP By vigane_vorm
        )
        WHERE count > 1
    )
    """
)

spurious_misspellings = (DataFrame(result.fetchall(), columns=['vigane_vorm', 'vorm', 'kaal'])
                         .groupby('vigane_vorm', as_index=False)
                         .aggregate(võimalikud_vasted=('vorm', lambda x:list(set(x.str.lower()))))
                         .assign(vastete_arv=lambda df: df['võimalikud_vasted'].map(len))
                         .pipe(lambda df: df[df['vastete_arv'] > 1])
                         .sort_values(['vastete_arv'], ascending=False))

display(spurious_misspellings)

,vigane_vorm,võimalikud_vasted,vastete_arv
0,el,"[eel, elu]",2
2,er,"[era, eri]",2
4,koku,"[kogu, kokku]",2
5,usu,"[uus, suu]",2
6,uu,"[uus, suu]",2
7,vaa,"[vaba, vara]",2
